# DeepSponsorBlock : Détection de séquences sponsorisées Youtube

test

## Imports

In [ ]:
import numpy as np
import pandas as pd

try: 
  import yt_dlp
except:
  ! pip install yt_dlp
  import yt_dlp

!apt -y install ffmpeg

from multiprocessing import cpu_count
from multiprocessing.pool import ThreadPool

from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

# Besoin de créer un raccourci vers le dossier partagé dans MON DRIVE avant, renommé DeepSponsorBlock SINON RIEN NE FONCTIONNE
base_dir = "/content/gdrive/MyDrive/DeepSponsorBlock/"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.
Mounted at /content/gdrive/


## Étude et nettoyage du dataset

In [ ]:
df=pd.read_csv(base_dir + "sponsorTimes.csv")

In [ ]:
df

In [ ]:
df.shape

(13316947, 20)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13316947 entries, 0 to 13316946
Data columns (total 20 columns):
 #   Column          Dtype  
---  ------          -----  
 0   videoID         object 
 1   startTime       float64
 2   endTime         float64
 3   votes           int64  
 4   locked          int64  
 5   incorrectVotes  int64  
 6   UUID            object 
 7   userID          object 
 8   timeSubmitted   int64  
 9   views           int64  
 10  category        object 
 11  actionType      object 
 12  service         object 
 13  videoDuration   float64
 14  hidden          int64  
 15  reputation      float64
 16  shadowHidden    int64  
 17  hashedVideoID   object 
 18  userAgent       object 
 19  description     object 
dtypes: float64(4), int64(7), object(9)
memory usage: 2.0+ GB


In [ ]:
df.describe

<bound method NDFrame.describe of               videoID   startTime     endTime  votes  locked  incorrectVotes  \
0         FfgT6zx4k3Q   446.51013   513.39233    225       0               1   
1         fBxtS9BpVWs    41.00000    53.00000    116       0               1   
2         AM1-ecnQsm4   164.88225   206.00000     -2       0               1   
3         fBxtS9BpVWs   714.00000   763.00000     64       0               1   
4         uqKGREZs6-w   475.52167   532.20874    302       0               1   
...               ...         ...         ...    ...     ...             ...   
13316942  aBqtEJv1W9k   508.18000   516.99500      0       0               1   
13316943  aBqtEJv1W9k  2091.88000  2104.00000      0       0               1   
13316944  A-hupIpyR0c     0.00000    12.32000      0       0               1   
13316945  A-hupIpyR0c  2441.74000  2448.02400      0       0               1   
13316946  W8ZjrcTbiq4     1.15500    64.99400      0       0               1   

     

In [ ]:
df.isna().sum()

videoID                 54
startTime                0
endTime                  0
votes                    0
locked                   0
incorrectVotes           0
UUID                     0
userID                   0
timeSubmitted            0
views                    0
category                 0
actionType               0
service                  0
videoDuration            0
hidden                   0
reputation              80
shadowHidden             0
hashedVideoID            0
userAgent          2493266
description       13295429
dtype: int64

In [ ]:
df.isna().sum().sum()

15788829

In [ ]:
len((df['videoID']).unique())

5693471

In [ ]:
df=df.drop(['description','userAgent'], axis=1)

In [ ]:
df=df.query("category=='sponsor'")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2228411 entries, 0 to 13316942
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   videoID         object 
 1   startTime       float64
 2   endTime         float64
 3   votes           int64  
 4   locked          int64  
 5   incorrectVotes  int64  
 6   UUID            object 
 7   userID          object 
 8   timeSubmitted   int64  
 9   views           int64  
 10  category        object 
 11  actionType      object 
 12  service         object 
 13  videoDuration   float64
 14  hidden          int64  
 15  reputation      float64
 16  shadowHidden    int64  
 17  hashedVideoID   object 
dtypes: float64(4), int64(7), object(7)
memory usage: 387.5+ MB


In [ ]:
df.shape

(2228411, 18)

In [ ]:
df.dropna(axis=0).isna().sum()

In [ ]:
len(df.dropna(axis=0))

39

In [ ]:
df_vip=pd.read_csv(base_dir + "vipUsers.csv")

In [ ]:
if(df_vip['userID'][0] in df['userID']):
  print('okay')
else: print("no")

no


In [ ]:
(df['userID'][0:10])

0    38e7c2af-09f4-4492-bf49-75e443962ccd
1    38e7c2af-09f4-4492-bf49-75e443962ccd
2    38e7c2af-09f4-4492-bf49-75e443962ccd
3    38e7c2af-09f4-4492-bf49-75e443962ccd
4    38e7c2af-09f4-4492-bf49-75e443962ccd
5    38e7c2af-09f4-4492-bf49-75e443962ccd
6    38e7c2af-09f4-4492-bf49-75e443962ccd
7    38e7c2af-09f4-4492-bf49-75e443962ccd
8    38e7c2af-09f4-4492-bf49-75e443962ccd
9    38e7c2af-09f4-4492-bf49-75e443962ccd
Name: userID, dtype: object

In [ ]:
df1=df.query("(locked>=1 and videoDuration<1200) or (votes>=10 and videoDuration<1200 and locked==0)") 
len(df1)

90230

In [ ]:
df1=df1.query("locked>=1 and reputation==0") 
len(df1)

9264

In [ ]:
df1.isna().sum()

videoID           0
startTime         0
endTime           0
votes             0
locked            0
incorrectVotes    0
UUID              0
userID            0
timeSubmitted     0
views             0
category          0
actionType        0
service           0
videoDuration     0
hidden            0
reputation        0
shadowHidden      0
hashedVideoID     0
dtype: int64

In [ ]:
(df['userID']=='57ddecc5b36813ddb8ea1eba73342c8a783527b884b6ebcb177bf37cafce7620').sum()

1415

In [ ]:
df

,videoID,startTime,endTime,votes,locked,incorrectVotes,UUID,userID,timeSubmitted,views,category,actionType,service,videoDuration,hidden,reputation,shadowHidden,hashedVideoID
0,FfgT6zx4k3Q,446.51013,513.39233,225,0,1,96150fa0-a28a-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,3221,sponsor,skip,YouTube,0.000,1,0.000000,0,0bfebefdc667735b19d5f2630e7e83a5f7fdddb880534e...
1,fBxtS9BpVWs,41.00000,53.00000,116,0,1,b2465943-1313-449c-b75c-08b14756ac0a,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,619,sponsor,skip,YouTube,0.000,0,0.000000,0,bdd81b2b8192683242fe3608c45d5b958ddc71e9b2981a...
2,AM1-ecnQsm4,164.88225,206.00000,-2,0,1,467137b0-af20-11e9-93c3-13d71c2776be,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,133,sponsor,skip,YouTube,0.000,0,0.000000,0,e294b5c58ceb6e4fd82919cc3387e60ac2bc9573570e26...
3,fBxtS9BpVWs,714.00000,763.00000,64,0,1,7c9619bd-8d91-4b42-a9be-39ffc99dad4a,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,326,sponsor,skip,YouTube,0.000,0,0.000000,0,bdd81b2b8192683242fe3608c45d5b958ddc71e9b2981a...
4,uqKGREZs6-w,475.52167,532.20874,302,0,1,622f9270-a2a1-11e9-b210-99c885575bb9,38e7c2af-09f4-4492-bf49-75e443962ccd,1564088876715,12428,sponsor,skip,YouTube,0.000,0,0.000000,0,70f8d0e75affa202ab510bd86828080af1dddf9218be46...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13316928,zyCaDmexT9U,3.42800,21.07000,0,0,1,2099b8a6854b56bafce061b1713b30fd19d21e4ab9e220...,0719afa780fc1a5163ba7a91083139247d1e7f47001ce2...,1674370589728,0,sponsor,skip,YouTube,4097.021,0,0.000000,0,de5ca36f99472672da79fb309bf7dcdb68610eafd3354a...
13316929,zyCaDmexT9U,578.43800,605.71500,0,0,1,2064917f02d059d3ca829d3135b88277b93561347ff1e9...,0719afa780fc1a5163ba7a91083139247d1e7f47001ce2...,1674370589728,0,sponsor,skip,YouTube,4097.021,0,0.000000,0,de5ca36f99472672da79fb309bf7dcdb68610eafd3354a...
13316931,0i84SlZWHWc,330.65600,465.50300,0,0,1,7109a12ee6920e701e5fbca7dcfb58020427211f9b4a00...,1f8052e3c092af9030f652066ebf3fe08f42908dbd58a6...,1674370644379,0,sponsor,skip,YouTube,3479.000,0,0.000000,0,fa6db790728f22501ebf6cf174350d0347fc64456b8132...
13316936,dSa4oL1_f5A,133.94400,197.04500,0,1,1,0a2f61134120884f22e9a660955d9c876bef2ba12dfe3d...,a5fa6d15d60fa762a18045826ca149bbacda3b30b9ed37...,1674370726935,0,sponsor,skip,YouTube,809.141,0,21.834482,0,fb658223dba48f5d29a52a98cb271980c4b00e30953b8b...


## Youtube-dl

L’outil permet de télécharger une vidéo youtube via une commande du type ``youtube-dl [OPTIONS] URL [URL...]``

Il existe une bibliothèque python permettant de l’utiliser dans un programme, ce que nous avons fait.

Les options sont nombreuses, mais les plus importantes dans le cadre de ce projet sont les suivantes
- ``writedescription`` : permet de télécharger la description de la vidéo en plus de son contenu
- ``format`` : contrôle le format du fichier téléchargé ainsi que sa qualité

L’outil télécharge les vidéos de façon séquentielle ce qui pose problème car le temps de téléchargement est limité par youtube à environ la durée de la vidéo. Ainsi, dans notre cas, il faut trouver un moyen de télécharger plusieurs vidéos en même temps puisque autrement le téléchargement d’une grosse quantité de vidéos sera extrêmement long.

Nous avons alors étudié la possibilité d’utiliser plusieurs coeurs du processeurs afin de télécharger les vidéos en parallèle ce qui semble être une solution efficace. Afin de réduire davantage le temps de téléchargement , on télécharge aussi la vidéo dans sa qualité la plus basse possible.

In [ ]:
def dl_videos(urls):
    ydl_opts = {
        'writedescription': True,
        'outtmpl': f'{base_dir}youtube-dl/%(id)s.mp4',
        'format': 'worstvideo[ext=mp4]+worstaudio[ext=m4a]',
        'ignoreerrors': 'only_download'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download(urls)


def dl_videos_parallel(urls):
    n_cpu = cpu_count()
    urls_split = np.array_split(urls, n_cpu)

    res = ThreadPool(n_cpu - 1).map(dl_videos, urls_split)

In [ ]:
print(cpu_count())

2


In [ ]:
urls = [
"https://www.youtube.com/watch?v=IYLTc3tGdzc",
"https://www.youtube.com/watch?v=2rt2g_afiAg",
"https://www.youtube.com/watch?v=BC6rvbxdywg",
"https://www.youtube.com/watch?v=qYyaKNvcKEA",
"https://www.youtube.com/watch?v=kLKEZBI3_gU",
"https://www.youtube.com/watch?v=Zi_XLOBDo_Y",
"https://www.youtube.com/watch?v=Y7-vP7TnluY"
]

dl_videos_parallel(urls)

[youtube] Extracting URL: https://www.youtube.com/watch?v=IYLTc3tGdzc
[youtube] IYLTc3tGdzc: Downloading webpage
[youtube] IYLTc3tGdzc: Downloading android player API JSON
[info] IYLTc3tGdzc: Downloading 1 format(s): 394+139
[info] Writing video description to: /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/IYLTc3tGdzc.description
[download] Destination: /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/IYLTc3tGdzc.f394.mp4
[download] 100% of  680.79KiB in 00:00:00 at 6.24MiB/s   
[download] Destination: /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/IYLTc3tGdzc.f139.m4a
[download] 100% of  798.34KiB in 00:00:00 at 6.87MiB/s   
[Merger] Merging formats into "/content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/IYLTc3tGdzc.mp4"
Deleting original file /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/IYLTc3tGdzc.f139.m4a (pass -k to keep)
Deleting original file /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/IYLTc3tGdzc.f394.mp4 (pass -k to keep)
[youtube] Extracting URL

ERROR: [youtube] qYyaKNvcKEA: Video unavailable. The uploader has not made this video available in your country


[youtube] Extracting URL: https://www.youtube.com/watch?v=kLKEZBI3_gU
[youtube] kLKEZBI3_gU: Downloading webpage
[youtube] kLKEZBI3_gU: Downloading android player API JSON
[info] kLKEZBI3_gU: Downloading 1 format(s): 394+139
[info] Writing video description to: /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/kLKEZBI3_gU.description
[download] Destination: /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/kLKEZBI3_gU.f394.mp4
[download] 100% of  169.24KiB in 00:00:00 at 2.14MiB/s   
[download] Destination: /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/kLKEZBI3_gU.f139.m4a
[download] 100% of  399.50KiB in 00:00:00 at 5.72MiB/s   
[Merger] Merging formats into "/content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/kLKEZBI3_gU.mp4"
Deleting original file /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/kLKEZBI3_gU.f394.mp4 (pass -k to keep)
Deleting original file /content/gdrive/MyDrive/DeepSponsorBlock/youtube-dl/kLKEZBI3_gU.f139.m4a (pass -k to keep)
[youtube] Extracting URL